In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import pathlib

python_path = pathlib.Path('.').absolute().parent/'python'
os.sys.path.insert(1, str(python_path))

In [3]:
## This is demo for kuka reaching a with mpc and diff_qp
## Author : Avadesh Meduri
## Date : 25/02/2022

import time
import numpy as np
import pinocchio as pin
from robot_properties_kuka.config import IiwaConfig
from matplotlib import pyplot as plt

import meshcat
import meshcat.transformations as tf
import meshcat.geometry as g

from vocam.diff_pin_costs import DiffFrameTranslationCost, DiffFrameVelocityCost
from vocam.inverse_qp import IOC

import torch
from torch.autograd import Function
from torch.nn import functional as F

In [4]:
robot = IiwaConfig.buildRobotWrapper()
model, data = robot.model, robot.data
f_id = model.getFrameId("EE")
j_id = [f_id]
for i in range(3, model.nq+1):
    j_id.append(model.getFrameId("A" + str(i)))

In [5]:
viz = pin.visualize.MeshcatVisualizer(robot.model, robot.collision_model, robot.visual_model)
viz.initViewer(open=False)
viz.loadViewerModel()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [6]:
dtc = DiffFrameTranslationCost.apply
dvc = DiffFrameVelocityCost.apply

def quadratic_loss(q_pred, x_des, nq, n_col):
    loss = 6.0e1*torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, f_id) - x_des)
    loss += 2.5e0*torch.linalg.norm(dvc(q_pred[-2*nq:], torch.zeros(nq), model, data, f_id)) # asking for zero velocity
    loss += 1e-2*torch.linalg.norm(q_pred[-2*nq:-nq]) # joint regularization
    
    for i in range(n_col):    
        loss += 2e0*torch.linalg.norm(dtc(q_pred[(3*i)*nq: (3*i+2)*nq], model, data, f_id) - x_des)
        loss += 5e-1*torch.linalg.norm(dvc(q_pred[(3*i)*nq: (3*i+2)*nq], q_pred[(3*i+2)*nq:(3*i+3)*nq], model, data, f_id)) # asking for zero velocity
        loss += 1e-2*torch.linalg.norm(q_pred[(3*i+2)*nq: (3*i+3)*nq]) # control regularization
        loss += 2e-1*torch.linalg.norm(q_pred[(3*i+1)*nq: (3*i+2)*nq]) # velocity regularization
        loss += 3e-3*torch.linalg.norm(q_pred[(3*i)*nq: (3*i+1)*nq]) # joint regularization
        loss += 2e-3*torch.linalg.norm(q_pred[(3*i)*nq+3: (3*i+1)*nq])
        loss += 4e-3*torch.linalg.norm(q_pred[(3*i)*nq+5])
        
        if i < n_col - 1:
            loss += 5e-2*torch.linalg.norm(torch.subtract(q_pred[(3*i+2)*nq: (3*i+3)*nq], \
                                                          q_pred[(3*i+5)*nq: (3*i+6)*nq]))

    return loss

In [12]:
def obstacle_loss(q_pred, x_des, nq, n_col, obs_pos_arr, obs_rad):
    
    loss = quadratic_loss(q_pred, x_des, nq, n_col)
    
    for k in range(len(obs_pos_arr)):
        obs_pos = obs_pos_arr[k]
        for i in range(len(j_id)):
            dist_loss = torch.linalg.norm(dtc(q_pred[-2*nq:], model, data, j_id[i]) - obs_pos) 
            loss += 1.0e2*torch.exp(-(dist_loss-obs_rad)/0.01)

    return loss

In [21]:
nq = model.nq
nv = model.nv

n_col = 5
u_max = [3.5,4.5,2.5, 2.5, 1.5, 1.5, 1.0]

lr = 1e-1
eps = 80

# q_des = np.hstack(((np.pi/4)*(np.random.randint(0, 2, size = 5)), np.zeros(2)))

q_des_arr = np.array([[2.1789238e-02,  3.3214998e-01, -1.4518893e-04, -8.7141126e-01,
                          6.0329604e-01, -1.3965217e-03,  1.4794523e-04],
                      [1.3737, 0.9711, 1.6139, 1.2188, 1.5669, 0.1236, 0.2565]])
x_des_arr = torch.tensor([[0.4, -0.3, 0.5], [0.4, 0.3, 0.5]]) #torch.tensor([[0.6, 0.4, 0.5]]) #[0.6, 0.4, 1.0]

# q_init = np.hstack(((np.pi/8)*np.random.choice(list(range(-2, -1)) + list(range(1, 3)), size=(4)), np.zeros(3)))

x_init = np.zeros(2*nq)
x_init[0:nq] = q_des_arr[0] + 0.3*2*(np.random.rand(nq) - 0.5)
x_init[0] -= 2*0.5*(np.random.rand(1) - 0.5)
x_init[2] -= 2*0.3*(np.random.rand(1) - 0.5)
x_init[nq:] = 0.7*2*(np.random.rand(nv) - 0.5)

x_in = x_init

In [442]:
r = 0.3*np.random.rand(1) +0.5
rint = np.random.randint(2)
theta = rint*(0.1*np.pi*(np.random.rand(1)) + 0.65*np.pi) + (1-rint)*(0.1*np.pi*(np.random.rand(1)) + 0.25*np.pi)
x_des = torch.squeeze(torch.tensor([r*np.sin(theta), r*np.cos(theta), 0.15*np.random.rand(1)+0.15]))

viz.viewer["box"].set_object(g.Sphere(0.05), 
                         g.MeshLambertMaterial(
                             color=0xff22dd,
                             reflectivity=0.8))
viz.viewer["box"].set_transform(tf.translation_matrix(x_des.detach().numpy()))


nb_balls = 1
obs_rad = 0.5
# obs_pos_arr = [torch.tensor([0.5, 0.0, 0.6]), torch.tensor([0.5, 0.3, 0.4]), torch.tensor([0.5, -0.3, 0.4])]
obs_pos_arr = []
for l in range(nb_balls):
    r = 0.0*np.random.rand(1) +0.5
    theta = 0.0*np.pi*(np.random.rand(1) - 0.5) + 0.5*np.pi
    ht = 0.0*np.random.rand(1) - 0.2
    obs_pos = torch.squeeze(torch.tensor([r*np.sin(theta), r*np.cos(theta), ht]))
    obs_pos_arr.append(obs_pos)
    viz.viewer["obstacle_" + str(l)].set_object(g.Sphere(obs_rad), 
                             g.MeshLambertMaterial(
                                 color=0x22ddff,
                                 reflectivity=0.8))
    viz.viewer["obstacle_" + str(l)].set_transform(tf.translation_matrix(obs_pos.detach().numpy()))
plt_ddq = []

for j in range(10):

    ioc = IOC(n_col, nq, u_max, 0.05, eps = 1.0, isvec=True)
    optimizer = torch.optim.Adam(ioc.parameters(), lr=lr)
    
    
    i = 0
    loss = 1000.
    old_loss = 10000.
    
    while loss > 0.03 and i < eps and abs(old_loss - loss) > 5e-5:
        x_pred = ioc(x_in) 

        old_loss = loss

#         loss = quadratic_loss(x_pred, x_des, nq, n_col)
        loss = obstacle_loss(x_pred, x_des, nq, n_col, obs_pos_arr, obs_rad)
        print("Index :" + str(i) + " loss is : " + str(loss.detach().numpy()), end = '\r', flush = True)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    x_pred = ioc(x_in).detach().numpy()

    for i in range(n_col+1):
        q = x_pred[3*nq*i:3*nq*i + nq]
        dq = x_pred[3*nq*i + nq:3*nq*i + 2*nq]
        if i < n_col:
            ddq = x_pred[3*nq*i + 2*nq:3*nq*i + 3*nq]
            plt_ddq.append(ddq)
        pin.forwardKinematics(model, data, q, dq, np.zeros(nv))
        pin.updateFramePlacements(model, data)

        viz.display(q)
        time.sleep(0.1)
    
    x_in = torch.tensor(x_pred[-2*nq:])

KeyboardInterrupt: 